In [43]:
# Will reload modeules after this when they change!
%load_ext autoreload
%autoreload 2

In [44]:
import pandas as pd
import numpy as np

In [ ]:
# Copy spotting datasets to beta:
from clone_ds_to_beta import copy_beta

spotted_ds = ['2020-03-12_17h55m21s', 
              '2020-03-15_19h57m37s',
              '2020-01-29_17h04m12s',
              '2020-02-06_09h48m17s',
              '2020-03-03_10h05m06s',
              '2019-05-28_13h26m26s',
              '2019-05-28_17h27m37s',
              '2019-05-29_16h19m01s',
              '2019-05-31_12h00m59s'
             ]
out_dict_list = []
for ds in spotted_ds:
    x = copy_beta(ds, 'HNaKM', 'core_metabolome_v3')
    out_dict_list.append(x)
    
out_df = pd.DataFrame(out_dict_list)
out_df.to_pickle('dsid_processing/spotting_cm3_df.pickle')

In [4]:
# Dataframe describing spotting experiment
out_df = pd.read_pickle('dsid_processing/spotting_cm3_df.pickle')
out_df = out_df.merge(pd.read_csv('dsid_processing/2020_May_13_Spotting_w_Ids.csv'),
                      on='ds_id_in')

In [5]:
out_df['paths'] = ['dsid_processing/2020-05-13_17h50m21s.csv', 'dsid_processing/2020-05-13_17h50m25s.csv', 
                   'dsid_processing/2020-05-13_17h50m28s.csv', 'dsid_processing/2020-05-13_17h50m31s.csv', 
                   'dsid_processing/2020-05-13_17h50m34s.csv', 'dsid_processing/2020-05-13_17h50m37s.csv', 
                   'dsid_processing/2020-05-13_17h50m41s.csv', 'dsid_processing/2020-05-13_17h50m45s.csv', 
                   'dsid_processing/2020-05-13_17h50m48s.csv']

In [6]:
a_id_list = ['HMDB0007865', 'HMDB0000610', 'HMDB0000067', 
             'HMDB0000878', 'HMDB0010382', 'HMDB0008578',
             'HMDB0008036', 'HMDB0008578', 'HMDB0010169',
             'HMDB0005432', 'HMDB0005393'
            ]
b_id_list = ['HMDB0000034', 'HMDB0000034', 'HMDB0001341', 
            'HMDB0000208', 'HMDB0000045', 'HMDB0000646', 
            'HMDB0000517', 'HMDB0000044', 'HMDB0000538',
            'HMDB0000030', 'HMDB0001847', 'HMDB0000094', 
            'HMDB0000904', 'HMDB0000562', 'HMDB0000574', 
            'HMDB0000192', 'HMDB0000630', 'HMDB0000121', 
            'HMDB0001058', 'HMDB0000112', 'HMDB0001254',
            'HMDB0000122', 'HMDB0001401', 'HMDB0000148', 
            'HMDB0000641', 'HMDB0000661', 'HMDB0000714',
            'HMDB0000197', 'HMDB0002092', 'HMDB0000190',
            'HMDB0000687', 'HMDB0000163', 'HMDB0001389',
            'HMDB0000902', 'HMDB0001406', 'HMDB0000214', 
            'HMDB0003337', 'HMDB0000220', 'HMDB0000159',
            'HMDB0001257', 'HMDB0000254', 'HMDB0000289',
            ]
c_id_list = ['HMDB0000186', 'HMDB0000034', 'HMDB0001341', 
            'HMDB0000045', 'HMDB0000646', 'HMDB0000538', 
            'HMDB0000217', 'HMDB0000089', 'HMDB0000095', 
            'HMDB0000660', 'HMDB0000143', 'HMDB0000122', 
            'HMDB0003337', 'HMDB0000123', 'HMDB0000133', 
            'HMDB0001273', 'HMDB0000172', 'HMDB0002092', 
            'HMDB0000161', 'HMDB0000517', 'HMDB0000168', 
            'HMDB0000191', 'HMDB0000574', 'HMDB0000148', 
            'HMDB0000641', 'HMDB0000177', 'HMDB0000687', 
            'HMDB0000182', 'HMDB0000696', 'HMDB0000159', 
            'HMDB0000162', 'HMDB0000929', 'HMDB0000158', 
            'HMDB0000883', 'HMDB0000190', 'HMDB0000163', 
            'HMDB0000169', 'HMDB0000211', 'HMDB0000205', 
            'HMDB0000283', 'HMDB0000187', 'HMDB0000258', 
            'HMDB0000036', 'HMDB0000300', 'HMDB0000292', 
            'HMDB0000098'
           ]
            

In [16]:
def spotted_ids(metaspace_csv_path, cmpd_set):
    # Generates ID list for intersection of METASPACE MS1 ids and
    # known spotted compounds for METASPACE MS2 library creation
    x = str(list(pd.read_csv(metaspace_csv_path, 
                             skiprows=2).moleculeIds))
    x = x.replace("['","").replace("']","").replace("'","").split(', ')
    var_dict = {'A': a_id_list, 'B': b_id_list, 'C': c_id_list}
    return list(set(var_dict[cmpd_set]) & set(x))

In [19]:
out_df['hits'] = out_df.apply(lambda x: spotted_ids(x.paths, 
                                                    x.cmpd_set), axis=1)
out_df['n_hts'] = out_df.hits.apply(lambda x: len(x))

In [27]:
out_df = out_df.replace('pos', 'positive').replace('neg', 'negative')

In [37]:
# Paths to databases and output for next cell
limit_list = None
outname = ''
out_path = 'to_metaspace/'
polarity = None
expt_neg = 'test_output/exp_negative.pickle'
expt_pos = 'test_output/exp_positive.pickle'
theo_neg = 'test_output/theo_negative.pickle'
theo_pos = 'test_output/theo_positive.pickle'
ref_expt = 'databases/ref_expt_df.pickle'
ref_theo = 'databases/ref_theo_df.pickle'

In [38]:
# Generats METASPACE database from Sirius output files
from sirius_to_metaspace_db import primary_loop

out_df['db_path'] = out_df.apply(lambda x: primary_loop(x.hits,
                                                        x.ds_id_out,
                                                        out_path,
                                                        x.polarity,
                                                        expt_pos,
                                                        theo_pos,
                                                        expt_neg,
                                                        theo_neg,
                                                        ref_expt,
                                                        ref_theo
                                                       ), axis=1)

1 (9636, 10)
df_merge:  1000
df_merge:  2000
df_merge:  3000
df_merge:  4000
df_merge:  5000
df_merge:  6000
df_merge:  7000
df_merge:  8000
df_merge:  9000
2 (192003, 15)  1     190622
 22      1370
 38         6
-4          3
-10         1
 0          1
Name: dmass, dtype: int64
3 (191999, 15)
Observed ions: 
 1     190622
22      1370
38         6
0          1
Name: dmass, dtype: int64
4 (191991, 15)
Correct ionformulas: 
 True    191991
Name: good_mass_calc, dtype: int64
to_metaspace/2020-05-13_17h50m21s.csv
1 (7747, 10)
df_merge:  1000
df_merge:  2000
df_merge:  3000
df_merge:  4000
df_merge:  5000
df_merge:  6000
df_merge:  7000
2 (231634, 15) -1     231629
-7          3
-3          1
-12         1
Name: dmass, dtype: int64
3 (231634, 15)
Observed ions: 
 -1     231629
-7          3
-3          1
-12         1
Name: dmass, dtype: int64
4 (231634, 15)
Error! No  H  to increment by  -1  in X!
Error! No  H  to increment by  -1  in X!
Error! No  H  to increment by  -1  in X!
Error! N

dmass not known!
Correct ionformulas: 
 True    231557
Name: good_mass_calc, dtype: int64
to_metaspace/2020-05-13_17h50m34s.csv
1 (9636, 10)
df_merge:  1000
df_merge:  2000
df_merge:  3000
df_merge:  4000
df_merge:  5000
df_merge:  6000
df_merge:  7000
df_merge:  8000
df_merge:  9000
2 (192003, 15)  1     190622
 22      1370
 38         6
-4          3
-10         1
 0          1
Name: dmass, dtype: int64
3 (191999, 15)
Observed ions: 
 1     190622
22      1370
38         6
0          1
Name: dmass, dtype: int64
4 (191991, 15)
Correct ionformulas: 
 True    191991
Name: good_mass_calc, dtype: int64
to_metaspace/2020-05-13_17h50m37s.csv
1 (9636, 10)
df_merge:  1000
df_merge:  2000
df_merge:  3000
df_merge:  4000
df_merge:  5000
df_merge:  6000
df_merge:  7000
df_merge:  8000
df_merge:  9000
2 (192003, 15)  1     190622
 22      1370
 38         6
-4          3
-10         1
 0          1
Name: dmass, dtype: int64
3 (191999, 15)
Observed ions: 
 1     190622
22      1370
38         6
0

In [40]:
out_df['metaspace_n'] = [19, 31, 276, 358, 78, 286, 252, 197, 152]
out_df['metaspace_parent'] = [3, 1, 19, 27, 9, 33, 32, 24, 16]
out_df.to_pickle('to_metaspace/9x_spotted_out_df.pickle')

In [46]:
# Copy dataset to beta and search with custom METASPACE MSMS db
from clone_ds_to_beta import copy_beta
out_list = []
ds_dict = dict(zip(list(out_df.ds_id_in), list(out_df.ds_id_out)))
for ds_old, ds_new in ds_dict.items():
    x = copy_beta(ds_old, 'M', ds_new)
    out_list.append(x)
out_df['ds_id_msms'] = out_list

{'ds_id_in': '2020-03-12_17h55m21s', 'ds_id_out': '2020-05-14_16h32m01s'}
{'ds_id_in': '2020-03-15_19h57m37s', 'ds_id_out': '2020-05-14_16h32m04s'}
{'ds_id_in': '2020-01-29_17h04m12s', 'ds_id_out': '2020-05-14_16h32m07s'}
{'ds_id_in': '2020-02-06_09h48m17s', 'ds_id_out': '2020-05-14_16h32m10s'}
{'ds_id_in': '2020-03-03_10h05m06s', 'ds_id_out': '2020-05-14_16h32m14s'}
{'ds_id_in': '2019-05-28_13h26m26s', 'ds_id_out': '2020-05-14_16h32m16s'}
{'ds_id_in': '2019-05-28_17h27m37s', 'ds_id_out': '2020-05-14_16h32m19s'}
{'ds_id_in': '2019-05-29_16h19m01s', 'ds_id_out': '2020-05-14_16h32m22s'}
{'ds_id_in': '2019-05-31_12h00m59s', 'ds_id_out': '2020-05-14_16h32m26s'}


In [47]:
out_df

,ds_id_in,ds_id_out,polarity,n_cmpds,cmpd_set,paths,hits,n_hts,db_path,metaspace_n,metaspace_parent,ds_id_msms
0,2020-03-12_17h55m21s,2020-05-13_17h50m21s,positive,15,A,dsid_processing/2020-05-13_17h50m21s.csv,"[HMDB0010169, HMDB0005393, HMDB0010382, HMDB00...",5,1,19,3,"{'ds_id_in': '2020-03-12_17h55m21s', 'ds_id_ou..."
1,2020-03-15_19h57m37s,2020-05-13_17h50m25s,negative,15,A,dsid_processing/2020-05-13_17h50m25s.csv,"[HMDB0010169, HMDB0007865]",2,1,31,1,"{'ds_id_in': '2020-03-15_19h57m37s', 'ds_id_ou..."
2,2020-01-29_17h04m12s,2020-05-13_17h50m28s,positive,46,B,dsid_processing/2020-05-13_17h50m28s.csv,"[HMDB0000094, HMDB0000517, HMDB0000687, HMDB00...",22,1,276,19,"{'ds_id_in': '2020-01-29_17h04m12s', 'ds_id_ou..."
3,2020-02-06_09h48m17s,2020-05-13_17h50m31s,positive,46,B,dsid_processing/2020-05-13_17h50m31s.csv,"[HMDB0000661, HMDB0000094, HMDB0000517, HMDB00...",28,1,358,27,"{'ds_id_in': '2020-02-06_09h48m17s', 'ds_id_ou..."
4,2020-03-03_10h05m06s,2020-05-13_17h50m34s,negative,46,B,dsid_processing/2020-05-13_17h50m34s.csv,"[HMDB0000661, HMDB0001389, HMDB0000220, HMDB00...",10,1,78,9,"{'ds_id_in': '2020-03-03_10h05m06s', 'ds_id_ou..."
5,2019-05-28_13h26m26s,2020-05-13_17h50m37s,positive,49,C,dsid_processing/2020-05-13_17h50m37s.csv,"[HMDB0000211, HMDB0000098, HMDB0000517, HMDB00...",37,1,286,33,"{'ds_id_in': '2019-05-28_13h26m26s', 'ds_id_ou..."
6,2019-05-28_17h27m37s,2020-05-13_17h50m41s,positive,49,C,dsid_processing/2020-05-13_17h50m41s.csv,"[HMDB0000211, HMDB0000098, HMDB0000517, HMDB00...",33,1,252,32,"{'ds_id_in': '2019-05-28_17h27m37s', 'ds_id_ou..."
7,2019-05-29_16h19m01s,2020-05-13_17h50m45s,negative,49,C,dsid_processing/2020-05-13_17h50m45s.csv,"[HMDB0000211, HMDB0000098, HMDB0000517, HMDB00...",35,1,197,24,"{'ds_id_in': '2019-05-29_16h19m01s', 'ds_id_ou..."
8,2019-05-31_12h00m59s,2020-05-13_17h50m48s,negative,49,C,dsid_processing/2020-05-13_17h50m48s.csv,"[HMDB0000211, HMDB0000098, HMDB0000517, HMDB00...",24,1,152,16,"{'ds_id_in': '2019-05-31_12h00m59s', 'ds_id_ou..."


In [ ]:
Friday:
# Submit hq_df jobs to beta to run
# SpaceM to Andreas

Next:
# Code DL and score results: dataframe (patch Lachlan), images (works)
# Code new scoring functions

Future:
# To compare expt only versus expt/theo

In [49]:
list(out_df.ds_id_out)

['2020-05-13_17h50m21s',
 '2020-05-13_17h50m25s',
 '2020-05-13_17h50m28s',
 '2020-05-13_17h50m31s',
 '2020-05-13_17h50m34s',
 '2020-05-13_17h50m37s',
 '2020-05-13_17h50m41s',
 '2020-05-13_17h50m45s',
 '2020-05-13_17h50m48s']

In [51]:
list(out_df.ds_id_msms)

[{'ds_id_in': '2020-03-12_17h55m21s', 'ds_id_out': '2020-05-14_16h32m01s'},
 {'ds_id_in': '2020-03-15_19h57m37s', 'ds_id_out': '2020-05-14_16h32m04s'},
 {'ds_id_in': '2020-01-29_17h04m12s', 'ds_id_out': '2020-05-14_16h32m07s'},
 {'ds_id_in': '2020-02-06_09h48m17s', 'ds_id_out': '2020-05-14_16h32m10s'},
 {'ds_id_in': '2020-03-03_10h05m06s', 'ds_id_out': '2020-05-14_16h32m14s'},
 {'ds_id_in': '2019-05-28_13h26m26s', 'ds_id_out': '2020-05-14_16h32m16s'},
 {'ds_id_in': '2019-05-28_17h27m37s', 'ds_id_out': '2020-05-14_16h32m19s'},
 {'ds_id_in': '2019-05-29_16h19m01s', 'ds_id_out': '2020-05-14_16h32m22s'},
 {'ds_id_in': '2019-05-31_12h00m59s', 'ds_id_out': '2020-05-14_16h32m26s'}]

1. Write script to downlaod and process results
2. Download and process results